In [ ]:
import os
import json
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
import json
import time
import spacy
import re
nlp = spacy.load("en_core_web_sm")

#### Read prompting result file and preprocess the data

In [ ]:
file_path = "prompting_result_200.npy"
code_generation = np.load(file_path, allow_pickle=True)

In [ ]:
def splitfeedback(sample):
    feedback = sample['feedback']
    step_result = {}
    for step_feedback in feedback.split('\n    def ')[1:]:
        key = step_feedback.replace('()','').split(':')[0]
        ### previous setting of temp is 0.7, which may bring some random \n\n main
        code_step =  [_.lstrip() for _ in step_feedback.split('\n\n')[0].split('\n ')[1:]]
        step_result[key] = code_step
        
    return step_result

In [ ]:
for item in code_generation:
    item['step_result'] = splitfeedback(item)

#### get raw operation and parameters

In [ ]:
operation_set = set()
parameter_set = set()
operation_dict = {}
parameter_dict = {}
for index,sample in enumerate(code_generation):
        sample_result = sample['step_result']
        for sen,step in sample_result.items():
            for code in step:
                try:
                    operation = code.split(' = ')[1].split('(')[0].lstrip().lower()
                    operation_set.add(operation.lstrip().rstrip())
                    if operation == 'strain':
                                print(sen+":"+code)
                    operation_dict[operation] = operation_dict[operation] + 1 if operation in operation_dict.keys() else 1
                    parameter_list = code.split('(')[1].split(')')[0].split(',')
                    for step_parameter in parameter_list:
                        if '=' in step_parameter:
                            parameter = step_parameter.split('=')[0].lstrip()
                            if parameter == 'ingredient':
                                print(index)
                            parameter_set.add(parameter)
                            parameter_dict[parameter] = parameter_dict[parameter] + 1 if parameter in parameter_dict.keys() else 1
                except:
                    print('wrong at {} because {}'.format(index,code))

#### selected operation and parameter

In [ ]:
operation_dict = sorted(operation_dict.items(),key = lambda x:x[1],reverse = True)
parameter_dict = sorted(parameter_dict.items(),key = lambda x:x[1],reverse = True)

In [ ]:
Operation_Dict = {'add':['add','top'],
                 'serve':['serve'],
                 'stir':['stir'],
                 'cook':['cook','saute'],
                 'place':['place','transfer','put','spoon','arrange','line','set','set_aside','return','let_stand','strain'],
                 'bake':['bake','grill','toast','roast','broil'],
                 'mix':['mix','combine','pour','toss','blend','puree','cream','shake'],
                 'preheat':['preheat'],
                 'heat':['heat','melt','brown','microwave'],
                 'cool':['cool','refrigerate','reduce','chill','freeze'],
                 'remove':['remove'],
                 'cover':['cover','seal'],
                 'sprinkle':['sprinkle','whisk'],
                 'cut':['cut','divide','slice','break','chop','peel','dice'],
                 'boil':['boil'],
                 'beat' :['beat'],
                 'simmer':['simmer'],
                 'drain':['drain'],
                 'season': ['season'],
                 'shape' : ['shape','fold','roll','press','knead','flatten','rise'],
                 'garnish':['garnish'],
                 'fry':['fry'],
                 'fill':['fill'],
                 'spread':['rub','spread','dip','grease','coat','brush'],
                 'spray':['spray','drizzle','scatter'],
                 'filt':['sift','scoop'],
                 'crush':['crush','mash'],
                 'steam':['steam'],
                 'dissolve':['dissolve'],
                 'make':['make','process','check','store','turn','measure']
                 }

Parameter_list = ['how','container','time','temp','tool','size','amount','speed','salt','pepper']

In [ ]:
Operation_reverse_dict = {}
for key,value_list in Operation_Dict.items():
    for value in value_list:
        Operation_reverse_dict[value]=key

#### Transfer to Symbolic Triplem

In [ ]:
sample = code_generation[0]
step_result = sample['step_result']
step_triple = []
# (input , operation_dict, output)
for sen,step_func_list in step_result.items():
    for step_func in step_func_list:
        step_output = step_func.split(" = ")[0]
        operation_detail = {}
        step_operation = step_func.split(" = ")[1].split("(")[0].lower()
        operation_detail['orginal_step_operation']=step_operation
        step_operation = Operation_reverse_dict.get(step_operation,'make').capitalize()  
        operation_detail['step_operation']=step_operation
        function_detail = step_func.split('(')[1].split(')')[0]
        step_input = [_.strip() for _ in function_detail.split(', ') if "=" not in _]
        step_parameter = [_.strip() for _ in function_detail.split(', ') if "=" in _]
        step_parameter,_ = filterParameter(step_parameter)
        if _ is not None:
            step_input.append(_)
        operation_detail['step_parameter']=step_parameter
        step_triple.append((step_input,operation_detail,step_output))

In [ ]:
for item in code_generation[0]['instructions']:
    print(item['text'])

In [ ]:
step_triple

In [ ]:
original_input = []
for step in step_triple:
    for ingredient in step[0]:
        if not ingredient.replace('h',"").isdigit():
            original_input.append(ingredient)

In [ ]:
operation_list = []
for step in step_triple:
    func_dict = step[1]
    operation_list.append(func_dict['step_operation'])

# Utils

In [ ]:
def filterParameter(step_parameter):
    filter_parameter = []
    ### special for ingredient
    ingredient = None
    for item in step_parameter:
        parameter_name = item.split('=')[0]
        parameter_value = item.split('=')[1]
        if parameter_name in Parameter_list:
            filter_parameter.append((parameter_name,parameter_value))
        if parameter_name == 'until':
            filter_parameter.append(('how','until_'+parameter_value))
        if parameter_name == 'heat':
            filter_parameter.append(('temp',parameter_value+'_heat'))
        if parameter_name == 'ingredient':
            ingredient = parameter_value
    return filter_parameter,ingredient